In [1]:
import pandas as pd
import datetime
from sklearn import datasets

In [2]:
from evidently.ui.workspace.cloud import CloudWorkspace

from evidently.report import Report
from evidently.metric_preset import DataQualityPreset
from evidently.metric_preset import DataDriftPreset
from evidently.metrics import *
from evidently.test_suite import TestSuite
from evidently.tests import *
from evidently.test_preset import DataDriftTestPreset
from evidently.tests.base_test import TestResult, TestStatus

In [3]:
adult_data = datasets.fetch_openml(name="adult", version=2, as_frame="auto")
adult = adult_data.frame
adult_ref = adult[~adult.education.isin(["Some-college", "HS-grad", "Bachelors"])]
adult_prod = adult[adult.education.isin(["Some-college", "HS-grad", "Bachelors"])]

/Users/sakshamgulati/.local/share/virtualenvs/MLOps_Template-k_6XISXV/lib/python3.10/site-packages/sklearn/datasets/_openml.py:1022: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


In [23]:
from dotenv import load_dotenv
import os

load_dotenv()
#connecting to evidently cloud workspace using API token
ws = CloudWorkspace(
token=os.getenv("evi_api"),
url="https://app.evidently.cloud")

# #creating a project in the workspace
__project_name__="Test Project"
__team_id__="74fdd884-9679-4693-819b-a6695e652e25"

project = ws.create_project(__project_name__,team_id=__team_id__)
project.description = "My project description"
project.save()

Project(id=UUID('687924e5-85e8-4381-ad0b-f972ffd195bc'), name='Test Project', description='My project description', dashboard=DashboardConfig(name='Test Project', panels=[], tabs=[], tab_id_to_panel_ids={}), team_id=UUID('74fdd884-9679-4693-819b-a6695e652e25'), date_from=None, date_to=None, created_at=datetime.datetime(2024, 6, 10, 21, 45, 9, 133912))

In [21]:
import requests
import json
from tqdm import tqdm


def request_stock_price_hist(symbol, token, sample = False):
    if sample == False:
        q_string = 'https://www.alphavantage.co/query?function=TIME_SERIES_WEEKLY_ADJUSTED&symbol={}&outputsize=full&apikey={}'
    else:
        q_string = 'https://www.alphavantage.co/query?function=TIME_SERIES_WEEKLY_ADJUSTED&symbol={}&apikey={}'

    print("Retrieving stock price data from Alpha Vantage (This may take a while)...")
    r = requests.get(q_string.format(symbol, token))
    print("Data has been successfully downloaded...")
    date = []
    colnames = list(range(0, 6))
    df = pd.DataFrame(columns = colnames)
    print("Sorting the retrieved data into a dataframe...")
    for i in tqdm(r.json()['Weekly Adjusted Time Series'].keys()):
        date.append(i)
        row = pd.DataFrame.from_dict(r.json()['Weekly Adjusted Time Series'][i], orient='index').reset_index().T[1:]
        df = pd.concat([df, row], ignore_index=True)
    df.columns = ["open", "high", "low", "close", "adjusted close", "volume", "dividend amount"]
    df['date'] = date
    df = df.set_index('date')
    df = df.sort_index(ascending=True)
    df = df.apply(pd.to_numeric, errors='coerce')
    return df
ALPHA_VANTAGE_API_KEY=os.getenv("ALPHA_VANTAGE_API_KEY")
data=request_stock_price_hist('IBM', ALPHA_VANTAGE_API_KEY)

Retrieving stock price data from Alpha Vantage (This may take a while)...
Data has been successfully downloaded...
Sorting the retrieved data into a dataframe...


100%|██████████| 1284/1284 [00:01<00:00, 773.02it/s]


In [24]:
#fetch data from the API
data_report = Report(
       metrics=[
           DataQualityPreset(),
       ],
    )
data_report.run(reference_data=None, current_data=data)
ws.add_report(project.id, data_report)

In [ ]:
data_report = Report(
        metrics=[
            DataDriftPreset(stattest='psi', stattest_threshold='0.3'),
            DataQualityPreset(),
        ],
        timestamp=datetime.datetime.now(),
    )

data_report.run(reference_data=adult_ref, current_data=adult_prod.iloc[0 : 100, :])

### Need three things
1. Data/Feature Drift-  data schema changes,A stale data source,Unexpected inputs. 
2. Model Prediction Drift-Track the descriptive statistics of the model output (e.g., mean predicted values, standard deviation).
Apply statistical tests (e.g., Kolmogorov-Smirnov test, Chi-squared test) to compare the most recent model outputs with the older ones.
Use probability distance metrics to compare distributions (e.g., Wasserstein distance).
